In [ ]:
"""
Transportation problem solver (dynamic parsing from a single string)

- Paste your table into `data_str` (format like in the problem).
- The script parses origin (distribution center) names, destination (store) names,
  per-unit costs, demands, and supplies, then solves a linear program to minimize cost.
"""

import re
import numpy as np
from scipy.optimize import linprog

data_str = """
\tAtlanta\tLexington\tMilwaukee\tSalt Lake City\tDemand
Seattle\t2.15\t1.95\t1.70\t0.60\t5000
San Francisco\t2.10\t2.00\t1.85\t0.55\t15000
Las Vegas\t1.75\t1.70\t1.50\t0.35\t3900
Tucson\t1.50\t1.53\t1.41\t0.60\t3400
Denver\t1.20\t1.10\t0.95\t0.40\t4600
Charlotte\t0.65\t0.55\t0.40\t0.95\t7500
Minneapolis\t0.90\t0.60\t0.40\t1.00\t3000
Fayetteville\t0.80\t1.05\t0.95\t1.10\t8500
Birmingham\t0.35\t0.60\t0.70\t1.35\t3500
Orlando\t0.15\t0.50\t0.70\t1.60\t12000
Cleveland\t0.60\t0.25\t0.35\t1.60\t9000
Philadelphia\t0.50\t0.30\t0.40\t1.70\t18000
Supply\t39400\t36800\t14500\t15000
"""

def parse_table(s):
    # split into non-empty lines
    lines = [ln.rstrip() for ln in s.strip().splitlines() if ln.strip()]
    if not lines:
        raise ValueError("No data lines found.")

    # split helper: split on tabs or two+ spaces
    splitter = lambda line: re.split(r'\t+|\s{2,}', line.strip())

    # header
    hdr = splitter(lines[0])
    # first token could be empty or "": remove if it's the row label placeholder
    if hdr[0] == '' or hdr[0].lower() == '': 
        hdr = hdr[1:]
    # last column header is 'Demand' — DC names are hdr[:-1]
    if hdr[-1].lower() == 'demand':
        dc_names = hdr[:-1]
    else:
        # fallback: assume last token is demand label anyway
        dc_names = hdr[:-1]

    store_names = []
    demand = []
    cost_rows = []

    supply = None

    for ln in lines[1:]:
        parts = splitter(ln)
        if not parts:
            continue
        first = parts[0].strip().lower()
        if first == 'supply':
            # supply line: should have 1 + number_of_dcs tokens
            sup_vals = parts[1:1+len(dc_names)]
            supply = [float(x.replace(',','')) for x in sup_vals]
            break
        else:
            # store row: name, costs for each DC, demand
            # some store names include spaces but our splitter uses tabs or 2+ spaces so it's safe
            name = parts[0]
            cost_vals = parts[1:1+len(dc_names)]
            dem = parts[1+len(dc_names)]
            store_names.append(name)
            cost_rows.append([float(x) for x in cost_vals])
            demand.append(float(dem))

    if supply is None:
        raise ValueError("Supply line not found or malformed.")

    C = np.array(cost_rows)       # shape: (num_stores, num_dcs)
    # For transport matrix typically origins (DCs) x destinations (stores) — we'll transpose
    C = C.T                       # shape now (num_dcs, num_stores)
    supply = np.array(supply, dtype=float)
    demand = np.array(demand, dtype=float)

    return dc_names, store_names, C, supply, demand

def solve_transport(dc_names, store_names, C, supply, demand):
    num_dc, num_store = C.shape
    # Decision variables x_ij flattened by origin-major (dc 0 stores..., dc1 stores..., ...)
    m = num_dc * num_store
    c = C.flatten()  # objective coefficients (minimize)

    # Equality constraints for demand: for each store j, sum_i x_ij == demand_j
    A_eq = np.zeros((num_store, m))
    b_eq = demand.copy()
    for j in range(num_store):
        for i in range(num_dc):
            idx = i * num_store + j
            A_eq[j, idx] = 1.0

    # Inequality constraints for supply: for each origin i, sum_j x_ij <= supply_i
    A_ub = np.zeros((num_dc, m))
    b_ub = supply.copy()
    for i in range(num_dc):
        for j in range(num_store):
            idx = i * num_store + j
            A_ub[i, idx] = 1.0

    bounds = [(0, None)] * m

    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    if not res.success:
        raise RuntimeError("Solver failed: " + res.message)

    x = res.x.reshape((num_dc, num_store))
    total_cost = res.fun

    shipped_by_origin = x.sum(axis=1)
    shipped_by_dest = x.sum(axis=0)

    # identify which DCs operate at capacity: shipped == supply (within tolerance)
    tol = 1e-6
    at_capacity = [dc_names[i] for i in range(num_dc) if abs(shipped_by_origin[i] - supply[i]) <= max(tol, 1e-6*abs(supply[i]))]

    return {
        'total_cost': total_cost,
        'flow_matrix': x,
        'shipped_by_origin': shipped_by_origin,
        'shipped_by_dest': shipped_by_dest,
        'at_capacity': at_capacity
    }

if __name__ == "__main__":
    dc_names, store_names, C, supply, demand = parse_table(data_str)
    sol = solve_transport(dc_names, store_names, C, supply, demand)

    print("Minimum total shipping cost = ${:,.2f}".format(sol['total_cost']))
    print("\nDistribution centers at capacity:")
    if sol['at_capacity']:
        for name in sol['at_capacity']:
            print(" -", name)
    else:
        print(" - None")

    # Optional: show shipped totals per DC (rounded)
    print("\nShipped by origin (DC -> total shipped):")
    for name, shipped, cap in zip(dc_names, sol['shipped_by_origin'], supply):
        print(f" {name:12s}: shipped = {shipped:,.0f}   capacity = {cap:,.0f}")

    # Optional: show top flows (origin->destination) for quick inspection
    print("\nTop 10 nonzero shipments (origin -> destination : quantity, unit cost):")
    num_dc, num_store = C.shape
    flows = []
    for i in range(num_dc):
        for j in range(num_store):
            qty = sol['flow_matrix'][i, j]
            if qty > 1e-6:
                flows.append((qty, dc_names[i], store_names[j], C[i, j]))
    flows.sort(reverse=True, key=lambda t: t[0])
    for qty, origin, dest, unit_cost in flows[:10]:
        print(f" {origin} -> {dest}: {qty:,.0f} units at ${unit_cost:.2f}/unit")


In [ ]:
print(9)

# Question 2

| **Scenario**                                                                                         | **Correct Classification**    |
| ---------------------------------------------------------------------------------------------------- | ----------------------------- |
| a. Each serving of chili should contain a quarter-pound of beef.                                     | **Requirement**               |
| b. Customer demand for a cereal is not expected to exceed 800 boxes during the month.                | **Bound**                     |
| c. Cash available to invest in March equals Feb. accounts receivable plus Feb. 28 investment yields. | **Balance constraint**        |
| d. A can of premium nuts should have at least twice as many cashews as peanuts.                      | **Proportional relationship** |
| e. A warehouse has 3,500 units available to ship to customers.                                       | **Limitation**                |
| f. A call center needs at least 15 service representatives on Monday morning (single variable).      | **Bound**                     |
| g. An ice cream manufacturer has 40 dozen fresh eggs at the start of the production shift.           | **Limitation**                |


# Question 4

In [ ]:
# Requires: pulp (pip install pulp)
from pulp import LpProblem, LpVariable, LpMaximize, value, LpStatus

# Create LP problem (maximize profit)
prob = LpProblem("Pointe_Shoes_Production", LpMaximize)

# Decision variables: number of shoes of each model (continuous; change to cat='Integer' if integers required)
M1 = LpVariable("Model1", lowBound=0)
M2 = LpVariable("Model2", lowBound=0)
M3 = LpVariable("Model3", lowBound=0)

# Objective: maximize profit
prob += 51*M1 + 45*M2 + 41*M3, "Total_Profit"

# Material constraints
prob += 12*M1 + 10*M2 + 14*M3 <= 1205, "Cardstock"
prob += 24*M1 + 20*M2 + 15*M3 <= 2005, "Satin"
prob += 40*M1 + 40*M2 + 30*M3 <= 7505, "Plain_Fabric"
prob += 11*M1 + 11*M2 + 10*M3 <= 1000, "Leather"

# Solve
prob.solve()

# Results
print("Status:", LpStatus[prob.status])
m1 = M1.varValue; m2 = M2.varValue; m3 = M3.varValue
total_profit = value(prob.objective)
print(f"Optimal production: Model1 = {m1:.6f}, Model2 = {m2:.6f}, Model3 = {m3:.6f}")
print(f"Total Profit = ${total_profit:.2f}\n")

# Resource usage and slack
card_used = 12*m1 + 10*m2 + 14*m3
satin_used = 24*m1 + 20*m2 + 15*m3
plain_used = 40*m1 + 40*m2 + 30*m3
leather_used = 11*m1 + 11*m2 + 10*m3

card_slack = 1205 - card_used
satin_slack = 2005 - satin_used
plain_slack = 7505 - plain_used
leather_slack = 1000 - leather_used

def binding(slack):
    return abs(slack) < 1e-6

print("Resource usage and slack:")
print(f" Cardstock:      used = {card_used:.6f} / 1205.00 , slack = {card_slack:.6f} , binding = {binding(card_slack)}")
print(f" Satin:          used = {satin_used:.6f} / 2005.00 , slack = {satin_slack:.6f} , binding = {binding(satin_slack)}")
print(f" Plain Fabric:   used = {plain_used:.6f} / 7505.00 , slack = {plain_slack:.6f} , binding = {binding(plain_slack)}")
print(f" Leather:        used = {leather_used:.6f} / 1000.00 , slack = {leather_slack:.6f} , binding = {binding(leather_slack)}")
